In [23]:
import numpy as np
import sys

nQueens = 8
STOP_CTR = 28
MUTATE = 0.000001
MUTATE_FLAG = True
MAX_ITER = 100000
POPULATION = None

In [24]:

class BoardPosition:
	def __init__(self):
		self.sequence = None
		self.fitness = None
		self.survival = None
	def setSequence(self, val):
		self.sequence = val
	def setFitness(self, fitness):
		self.fitness = fitness
	def setSurvival(self, val):
		self.survival = val
	def getAttr(self):
		return {'sequence':self.sequence, 'fitness':self.fitness, 'survival':self.survival}

In [25]:
def fitness(chromosome = None):
	clashes = 0;
	row_col_clashes = abs(len(chromosome) - len(np.unique(chromosome)))
	clashes += row_col_clashes

	# calculate diagonal clashes
	for i in range(len(chromosome)):
		for j in range(len(chromosome)):
			if ( i != j):
				dx = abs(i-j)
				dy = abs(chromosome[i] - chromosome[j])
				if(dx == dy):
					clashes += 1
	return 28 - clashes	

In [26]:

def generateChromosome():
	# randomly generates a sequence of board states.
	global nQueens
	init_distribution = np.arange(nQueens)
	np.random.shuffle(init_distribution)
	return init_distribution

In [27]:
def generatePopulation(population_size = 100):
	global POPULATION

	POPULATION = population_size

	population = [BoardPosition() for i in range(population_size)]
	for i in range(population_size):
		population[i].setSequence(generateChromosome())
		population[i].setFitness(fitness(population[i].sequence))

	return population



In [28]:
def getParent():
	globals()	
	parent1, parent2 = None, None
	# parent is decided by random probability of survival.
	# since the fitness of each board position is an integer >0, 
	# we need to normaliza the fitness in order to find the solution
	
	summation_fitness = np.sum([x.fitness for x in population])
	for each in population:
		each.survival = each.fitness/(summation_fitness*1.0)

	while True:
		parent1_random = np.random.rand()
		parent1_rn = [x for x in population if x.survival <= parent1_random]
		try:
			parent1 = parent1_rn[0]
			break
		except:
			pass

	while True:
		parent2_random = np.random.rand()
		parent2_rn = [x for x in population if x.survival <= parent2_random]
		try:
			t = np.random.randint(len(parent2_rn))
			parent2 = parent2_rn[t]
			if parent2 != parent1:
				break
			else:
				print("equal parents")
				continue
		except:
			print("exception")
			continue

	if parent1 is not None and parent2 is not None:
		return parent1, parent2
	else:
		sys.exit(-1)



In [29]:
def reproduce_crossover(parent1, parent2):
	globals()
	n = len(parent1.sequence)
	c = np.random.randint(n, size=1)
	child = BoardPosition()
	child.sequence = []
	child.sequence.extend(parent1.sequence[0:c])
	child.sequence.extend(parent2.sequence[c:])
	child.setFitness(fitness(child.sequence))
	return child

In [30]:

def mutate(child):
	"""	
	- according to genetic theory, a mutation will take place
	when there is an anomaly during cross over state
	- since a computer cannot determine such anomaly, we can define 
	the probability of developing such a mutation 
	"""
	if child.survival < MUTATE:
		c = np.random.randint(8)
		child.sequence[c] = np.random.randint(8)
	return child

In [31]:
def GA(iteration):
	print(" #"*10 ,"Executing Genetic  generation : ", iteration , " #"*10)
	globals()
	newpopulation = []
	for i in range(len(population)):
		parent1, parent2 = getParent()
		# print "Parents generated : ", parent1, parent2

		child = reproduce_crossover(parent1, parent2)

		if(MUTATE_FLAG):
			child = mutate(child)

		newpopulation.append(child)
	return newpopulation

In [32]:
def stop():
	globals()
	fitnessvals = [pos.fitness for pos in population]
	if STOP_CTR in fitnessvals:
		return True
	if MAX_ITER == iteration:
		return True
	return False

In [33]:
column = [0, 1, 2, 3, 4, 5, 6, 7]
chess = [["*", "*", "*", "*", "*", "*", "*", "*"], 
         ["*", "*", "*", "*", "*", "*", "*", "*"],
         ["*", "*", "*", "*", "*", "*", "*", "*"],
         ["*", "*", "*", "*", "*", "*", "*", "*"],
         ["*", "*", "*", "*", "*", "*", "*", "*"],
         ["*", "*", "*", "*", "*", "*", "*", "*"],
         ["*", "*", "*", "*", "*", "*", "*", "*"],
         ["*", "*", "*", "*", "*", "*", "*", "*"],]

In [34]:
def printResult(sequence, chess=chess):
    for i in range(0, 8, 1):
        chess[sequence[i]][column[i]] = "Q"
    for c in chess:
        print(c, end="\n")

In [35]:
population = generatePopulation(10000)
iteration = 0;
while not stop():
	population = GA(iteration)
	iteration +=1 

for each in population:
	if each.fitness == 28:
		print(each.sequence.tolist())
		printResult(each.sequence.tolist())
		print("\n")
		break;

[1, 5, 7, 2, 0, 3, 6, 4]
['*', '*', '*', '*', 'Q', '*', '*', '*']
['Q', '*', '*', '*', '*', '*', '*', '*']
['*', '*', '*', 'Q', '*', '*', '*', '*']
['*', '*', '*', '*', '*', 'Q', '*', '*']
['*', '*', '*', '*', '*', '*', '*', 'Q']
['*', 'Q', '*', '*', '*', '*', '*', '*']
['*', '*', '*', '*', '*', '*', 'Q', '*']
['*', '*', 'Q', '*', '*', '*', '*', '*']


